In [1]:
# Importamos los packages necesarios 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#cargamos los dataset necesarios para el trabajo. 
df_demographics = pd.read_csv('data/data_raw/df_final_demo.txt', sep=',')
df_experiment = pd.read_csv('data/data_raw/df_final_experiment_clients.txt', sep=',')

#Carga de las dos partes 
df_webdata1 = pd.read_csv('data/data_raw/df_final_web_data_pt_1.txt', sep=',')
df_webdata2 = pd.read_csv('data/data_raw/df_final_web_data_pt_2.txt', sep=',')

In [ ]:
# Creamo las funcion para juntar los dos dataframes sobre el paso de los clientes en el proceso web.

def merge_dataframes(df1, df2, key='client_id'):
    
    merged_df = pd.merge(df1, df2, how='inner', on=key)

    # Eliminar columnas duplicadas que terminan en '_y'
    cols_to_drop = [col for col in merged_df.columns if col.endswith('_y')]
    merged_df.drop(columns=cols_to_drop, inplace=True)

    # Renombrar columnas que terminan en '_x' eliminando el sufijo
    merged_df.rename(columns={
        col: col.replace('_x', '') for col in merged_df.columns if col.endswith('_x')
    }, inplace=True)
    
    # Eliminar duplicados y ordenar por client_id y date_time
    merged_df = merged_df.drop_duplicates().sort_values(['client_id', 'date_time'])

    return merged_df

# Llamamos a la funcion para hacer el merge de los datos 
df_final_webdata = merge_dataframes(df_webdata1, df_webdata2)

# se guarda en la carpeta files para usarlo en el futuro 
df_final_webdata.to_csv('data/df_final_web_data.txt', index=False)

#Cargamos el dataframe final para comprobar que se ha guardado correctamente. 
df_final_webdata = pd.read_csv('data/df_final_web_data.txt',sep=',')
df_final_webdata.head(10)

,client_id,visitor_id,visit_id,process_step,date_time
0,1643,910842861_51062379179,379946188_1773022140_107963,start,2017-04-15 15:06:00
1,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:36:07
2,1643,910842861_51062379179,633860590_96880450633_976109,step_1,2017-04-24 06:36:32
3,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:37:17
4,1643,910842861_51062379179,633860590_96880450633_976109,start,2017-04-24 06:38:20
5,1677,361240963_15830034505,494607202_74539107264_966618,start,2017-03-31 10:59:27
6,1677,361240963_15830034505,494607202_74539107264_966618,step_1,2017-03-31 10:59:41
7,1677,361240963_15830034505,494607202_74539107264_966618,step_2,2017-03-31 11:00:40
8,1677,361240963_15830034505,494607202_74539107264_966618,start,2017-03-31 11:01:50
9,1680,262688118_24224945474,807102410_81976926205_767113,start,2017-03-15 09:57:21


Teniendo los dataframes unidos pasamos a explorar el resto. 

In [6]:
# Limpiamos el dataframe demographics y lo exportamos ya limpio 

def limpiar_demograficos(df):
    # Normaliza los nombres de las columnas.
    df = df.rename(columns={
        'clnt_tenure_yr': 'client_tenure_years',
        'clnt_tenure_mnth': 'client_tenure_months',
        'clnt_age': 'age',
        'gendr': 'gender',
        'num_accts': 'number_of_accounts',
        'bal': 'balance',
    })
    # Elimina los valores nulos ya que no representan ni un 0.02% del total de los datos. 
    df = df.dropna()
    # Cambia X por U en la columna gender son muy pocos datos por lo que normalizamos. 
    df["gender"] = df["gender"].replace({"X": "U"})
    return df

df_demo_clean = limpiar_demograficos(df_demographics)
df_demo_clean.to_csv('data/df_demographic_clean.csv', sep=',',index=False)